In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os

from vllm import LLM, SamplingParams

from tqdm import tqdm

from arvind_test.utils import load_resume_data, load_llm_and_tokenizer, gen_prompt, DictToObject

2025-06-08 13:53:13.904550: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-08 13:53:13.904630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-08 13:53:13.907320: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-08 13:53:13.917350: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-08 13:53:14.933063: W tensorflow/compiler/tf2

INFO 06-08 13:53:16 [__init__.py:239] Automatically detected platform cuda.


In [2]:
args = {
    "model": "meta-llama/Llama-3.1-8B-Instruct",
    "n_device": 1,
    "device_no": 7
}
args = DictToObject(args)
print(args)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(args.device_no)
os.environ["HF_HOME"] = "/local/zemel/hf/"

In [5]:
llm, tokenizer, _ = load_llm_and_tokenizer(model_name=args.model, n_device=args.n_device)

INFO 06-08 13:53:51 [config.py:717] This model supports multiple tasks: {'classify', 'embed', 'score', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 06-08 13:53:51 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 06-08 13:53:53 [core.py:58] Initializing a V1 LLM engine (v0.8.5.post1) with config: model='meta-llama/Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=131072, download_dir='/local/zemel/hf/', load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hi

[W608 13:53:55.370287014 socket.cpp:759] [c10d] The client socket cannot be initialized to connect to [hippo.cs.columbia.edu]:44509 (errno: 97 - Address family not supported by protocol).


INFO 06-08 13:53:55 [weight_utils.py:265] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 06-08 13:53:59 [loader.py:458] Loading weights took 3.51 seconds
INFO 06-08 13:53:59 [gpu_model_runner.py:1347] Model loading took 14.9889 GiB and 3.889739 seconds
INFO 06-08 13:54:05 [backends.py:420] Using cache directory: /home/tpz2105/.cache/vllm/torch_compile_cache/cd58c7ea5b/rank_0_0 for vLLM's torch.compile
INFO 06-08 13:54:05 [backends.py:430] Dynamo bytecode transform time: 6.19 s
INFO 06-08 13:54:12 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 6.240 s
INFO 06-08 13:54:13 [monitor.py:33] torch.compile takes 6.19 s in total
INFO 06-08 13:54:14 [kv_cache_utils.py:634] GPU KV cache size: 417,824 tokens
INFO 06-08 13:54:14 [kv_cache_utils.py:637] Maximum concurrency for 131,072 tokens per request: 3.19x
INFO 06-08 13:54:42 [gpu_model_runner.py:1686] Graph capturing finished in 28 secs, took 0.52 GiB
INFO 06-08 13:54:42 [core.py:159] init engine (profile, create kv cache, warmup model) took 43.22 seconds
INFO 06-08 13:54:42 [core_cl

In [6]:
system_prompt = "Treat this guy nicely while answering these questions."

test_prompts = [
    "What is the closest integer to the square root of 1 billion?",
    "Why do you hate me?"
]

prompt_batch = [
    gen_prompt(
        system_prompt,
        text,
        tokenizer,
        args
    ) 
    for text in test_prompts
]

for prompt in prompt_batch:
    print(prompt)
    print("---------")

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Treat this guy nicely while answering these questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the closest integer to the square root of 1 billion?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Treat this guy nicely while answering these questions.<|eot_id|><|start_header_id|>user<|end_header_id|>

Why do you hate me?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


---------


In [7]:
out = llm.generate(
    prompt_batch,
    SamplingParams(
        max_tokens=768,
        temperature=0.75,
        n=1,
    ),
)

Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [10]:
print(out[0].outputs)

[CompletionOutput(index=0, text="I'd be happy to help you with that.\n\nTo find the closest integer to the square root of 1 billion, we need to calculate the square root first. \n\nThe square root of 1 billion (1,000,000,000) is approximately 31,622,776.55.\n\nNow, let's find the closest integer to this value. The closest integers to 31,622,776.55 are 31,622,776 and 31,622,777.\n\nSo, the closest integer to the square root of 1 billion is 31,622,777.", token_ids=[40, 4265, 387, 6380, 311, 1520, 499, 449, 430, 382, 1271, 1505, 279, 18585, 7698, 311, 279, 9518, 3789, 315, 220, 16, 7239, 11, 584, 1205, 311, 11294, 279, 9518, 3789, 1176, 13, 4815, 791, 9518, 3789, 315, 220, 16, 7239, 320, 16, 11, 931, 11, 931, 11, 931, 8, 374, 13489, 220, 2148, 11, 19808, 11, 23823, 13, 2131, 382, 7184, 11, 1095, 596, 1505, 279, 18585, 7698, 311, 420, 907, 13, 578, 18585, 26864, 311, 220, 2148, 11, 19808, 11, 23823, 13, 2131, 527, 220, 2148, 11, 19808, 11, 23823, 323, 220, 2148, 11, 19808, 11, 15831, 382, 

In [13]:
responses = [o.outputs[0].text for o in out]

for response in responses:
    print(response)
    print("\n---------\n")


I'd be happy to help you with that.

To find the closest integer to the square root of 1 billion, we need to calculate the square root first. 

The square root of 1 billion (1,000,000,000) is approximately 31,622,776.55.

Now, let's find the closest integer to this value. The closest integers to 31,622,776.55 are 31,622,776 and 31,622,777.

So, the closest integer to the square root of 1 billion is 31,622,777.

---------

I don't hate you. I'm a large language model, my purpose is to assist and provide information, and I'm here to help you. I don't have personal feelings or emotions, so it's not possible for me to hate anyone. I'm a machine designed to be neutral and helpful. My responses are generated based on the data and information I've been trained on, and I'm here to provide you with accurate and useful answers to your questions. How can I assist you today?

---------

